## Package Imports

In [1]:
import rioxarray
import xarray
import numpy as np

## Study Area

Nallurahalli is a locality in Whitefield which suffered significant flooding in Sep 2022. It was one of the disaster zones which was visited by the then CM Shri BS Yediyurappa.

Historically, this area has seen rapid development during recent years and has significant built-up and relatively less green cover due to which the surface run-off can accumulate due to less natural infiltration and swollen drains thereby causing floods.

There are mainly 3 internal roads which connect Nallurahalli to nearby highways.
- Borewell Road
- Siddapura Road
- Vydehi Road

Study area is taken around these pivotal roads so as to identify potential low-lying zones. If flooding happens around these areas it can affect the masses pretty bad and can cut it off from necessary help. 

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "output/img_study_area.png", width=500, height=500)

## Data Gathering

- Elevation Data is available as open-source from SRTM mission (Shuttle Radar Topography Mission)
- The resolution of the data is 30m (1 arc second)
- Sample data for the study region was downloaded from USGS 
- https://earthexplorer.usgs.gov/

## Read SRTM

In [ ]:
pt = './data/n12_e077_1arc_v3.tif'
xda = rioxarray.open_rasterio(pt)
xda

In [ ]:
xda.data.max(), xda.data.min()

In [ ]:
xda.data

## Raster Segmentation

In [ ]:
dtm_arr = xda.data[0]
dtm_arr

In [4]:
m,n = dtm_arr.shape

In [5]:
def nbr_hts(i,j):
    return dtm_arr[(max(i-1,0),max(j-1,0))],dtm_arr[(max(i-1,0),j)],dtm_arr[(max(i-1,0),min(j+1,n-1))],dtm_arr[(i,max(j-1,0))],dtm_arr[(i,min(j+1,n-1))],dtm_arr[(min(i+1,m-1),max(j-1,0))],dtm_arr[(min(i+1,m-1),max(j-1,0))],dtm_arr[(min(i+1,m-1),min(j+1,n-1))]

In [ ]:
nbr_hts(0,0)

In [7]:
def classify(i,j):
    hij = dtm_arr[i,j]
    return 1 if hij < min(nbr_hts(i,j)) else 0

In [ ]:
segmented = np.zeros((m,n))
for i in range(m):
    for j in range(n):
        print(i,j)
        segmented[i,j] = classify(i,j)

In [ ]:
np.unique(segmented,return_counts=True)

In [12]:
segmented_xda = xarray.DataArray(
    data=[segmented],
    dims=['band','y','x'],
    coords=xda.coords,
    attrs=dict(
        description="Low lying areas",
        units="boolean",
    )
)
# out = './output/segmented1.tiff'
# segmented_xda.rio.to_raster(out)

In [26]:
x = xda.x.values
y = xda.y.values
x, y = np.meshgrid(x, y)

In [ ]:
x.shape, y.shape, segmented.shape

In [32]:
x, y, segmented = x.flatten(), y.flatten(), segmented.flatten()

In [ ]:
import pandas as pd
df_seg = pd.DataFrame.from_dict({'label': segmented, 'x': x, 'y': y})
df_seg

In [ ]:
df_target = df_seg[df_seg.label == 1].reset_index(drop=True)
df_target

In [ ]:
df_target

In [ ]:
import geopandas as gpd
df_target_pts = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_xy(df_target['x'], df_target['y'], crs=xda.rio.crs))
df_target_pts = df_target_pts.to_crs('EPSG:4326')
df_target_pts

In [37]:
out = '/Users/gouher/Documents/personal/codes/geo/urban_flooding/output/low_lying_pts.shp'
df_target_pts.to_file(out)

In [39]:
out = '/Users/gouher/Documents/personal/codes/geo/urban_flooding/output/low_lying_pts.csv'
df_target.to_csv(out,index=False)

## Observation

- White pixels denote pockets of low-lying zones 
- Such zones have relatively lower elevation compared to the surrounding pixels
- During heavy rain, surface run-off can accumulate in these zones and cause flooding

In [ ]:
Image(url= "./output/img_segmented.png", width=500, height=500)

## Validation

- Red highlighted pixel above is the DNA Opulence Apartments which got flooded during Sep 2022 rains
- Similarly above it is SS Felicity Apartments which also got flooded heavily during the same time.
- The closeness of these apartments with nearby Storm Water Drain may also have played a part in the flooding.